In [1]:
print("hi")

hi


In [ ]:

import sys
import os
import torch

sys.path.append(os.getcwd())

try:
    from train_multichannel import train
    print("✅ Modulo 'train_multichannel' importato correttamente.")
except ImportError as e:
    print(f"❌ ERRORE: Non trovo 'train_multichannel.py'. Assicurati che sia nella cartella: {os.getcwd()}")
    raise e

print("=" * 80)
print(" TRAINING MULTI-CHANNEL MODEL")
print("=" * 80)

# Paths
JSON_FILE = r"/workspace/data/data/PopulationDataset/final_clustered_samples.json"
BASE_DIR = r"/workspace/data/data/PopulationDataset"

# Model configuration
MODEL_TYPE = 'baseline'        # 
PRETRAINED = True              # TODO CHANGE PRETEAI
FREEZE_BACKBONE = False        #

# Training hyperparameters
NUM_EPOCHS = 15
BATCH_SIZE = 24               # 
LEARNING_RATE = 1e-4
PATIENCE = 15                 # Early stopping patience

# Output paths
CHECKPOINT_DIR = 'checkpoints_multichannel'
LOG_DIR = 'runs/multichannel_baseline'

# Hardware
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_WORKERS = 4               # 

# ============================================================================
# ESECUZIONE
# ============================================================================

print("\nConfig:")
print(f"  Dataset: {BASE_DIR}")
print(f"  Model: {MODEL_TYPE} | Pretrained: {PRETRAINED}")
print(f"  Device: {DEVICE} | Batch size: {BATCH_SIZE}")
print(f"  Epochs: {NUM_EPOCHS} | LR: {LEARNING_RATE}")

if DEVICE == 'cpu':
    print("\n⚠️ WARNING: CPU")

try:
    print("\n" + "=" * 80)
    print(" 🚀 STARTING TRAINING")
    print("=" * 80)
    
    train(
        # Data
        json_file=JSON_FILE,
        base_dir=BASE_DIR,
        
        # Model
        model_type=MODEL_TYPE,
        pretrained=PRETRAINED,
        freeze_backbone=FREEZE_BACKBONE,
        
        # Training
        num_epochs=NUM_EPOCHS,
        batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        weight_decay=1e-4,
        
        # Early stopping
        patience=PATIENCE,
        
        # Checkpointing
        checkpoint_dir=CHECKPOINT_DIR,
        save_every=5,
        
        # Logging
        log_dir=LOG_DIR,
        
        # System
        num_workers=NUM_WORKERS,
        device=DEVICE,
        resume_from=None  # Esempio: 'checkpoints_multichannel/checkpoint_epoch_5.pth'
    )
    
    print("\n✅ TRAINING DONE!")
    print(f"Best model: {CHECKPOINT_DIR}/best_model.pth")

except KeyboardInterrupt:
    print("\n⚠️ Training interrotto manualmente.")
except Exception as e:
    print(f"\n❌ ERRORE: {e}")
    import traceback
    traceback.print_exc()

✅ Modulo 'train_multichannel' importato correttamente.
 TRAINING MULTI-CHANNEL MODEL

Configurazione Attuale:
  Dataset: /workspace/data/data/PopulationDataset
  Model: baseline | Pretrained: True
  Device: cuda | Batch size: 24
  Epochs: 15 | LR: 0.0001

 🚀 STARTING TRAINING
TRAINING BASELINE MODEL

Configuration:
  Device: cuda
  Model: baseline
  Epochs: 15
  Batch size: 24
  Learning rate: 0.0001
  Early stopping patience: 15

Loading Data

📊 Geographic Split:
  Train: 56 cities (26631 samples)
  Val:   12 cities (5857 samples)
  Test:  12 cities (5646 samples)

🏗️ Creating datasets...
📂 Loading JSON from /workspace/data/data/PopulationDataset/final_clustered_samples.json...
🔍 Indexing: found 80 RGB, 80 DEM, 79 LandUse files.
✅ Mapped 79 cities completely. (1 incomplete ignored)
💾 Pre-loading TIFF images...
   ✅ Loaded 56 cities into cache
📂 Loading JSON from /workspace/data/data/PopulationDataset/final_clustered_samples.json...
🔍 Indexing: found 80 RGB, 80 DEM, 79 LandUse files.
✅

Epoch 0 [Train]:  53%|█████▎    | 575/1076 [00:36<00:32, 15.62it/s, loss=1.69e+8]